<a href="https://colab.research.google.com/github/Tanuja8448/FMML-LAB/blob/main/Copy_of_FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

Answer 1:
         When you increase the percentage of the validation set:

1. *Accuracy may decrease*: With a larger validation set, the model has less training data, which can lead to lower accuracy.
2. *Overfitting detection improves*: A larger validation set provides a more accurate estimate of the model's performance, helping detect overfitting.
3. *Hyperparameter tuning improves*: A larger validation set allows for more reliable hyperparameter tuning, as the model's performance is evaluated on a larger, more diverse set.

When you reduce the percentage of the validation set:

1. *Accuracy may increase*: With a smaller validation set, the model has more training data, which can lead to higher accuracy.
2. *Overfitting detection worsens*: A smaller validation set may not accurately represent the model's performance, making it harder to detect overfitting.
3. *Hyperparameter tuning worsens*: A smaller validation set provides a less reliable estimate of the model's performance, making hyperparameter tuning more challenging.

In general, a validation set size of 15% to 30% of the total dataset is a good starting point. However, the optimal size depends on the specific problem, dataset, and model.

Here's a rough outline of the effects of validation set size on accuracy:

- *Small validation set (5-10%)*: High accuracy, but potentially overfitting and poor hyperparameter tuning.
- *Medium validation set (15-30%)*: Balanced accuracy, overfitting detection, and hyperparameter tuning.
- *Large validation set (40-50%)*: Lower accuracy, but improved overfitting detection and hyperparameter tuning.

Keep in mind that these are general guidelines, and the optimal validation set size depends on your specific use case.

Answer 2:
         The size of the train and validation sets significantly impacts the accuracy prediction on the test set using the validation set. Here's how:

*Train set size:*

- *Small train set*: The model may not learn enough patterns, leading to poor accuracy on both validation and test sets.
- *Large train set*: The model can learn more patterns, but may overfit, leading to poor accuracy on the test set despite good validation set accuracy.

*Validation set size:*

- *Small validation set*: May not accurately represent the test set, leading to:
    - Overestimation of accuracy (if validation set is too easy)
    - Underestimation of accuracy (if validation set is too hard)
- *Large validation set*: More accurately represents the test set, providing a better estimate of accuracy.

*Ideal scenario:*

- *Large train set* (enough data to learn patterns)
- *Medium to large validation set* (accurately represents test set, but not too large to waste data)

By having a large enough train set and a representative validation set, you can:

1. Train a model that generalizes well
2. Accurately estimate test set accuracy using the validation set

This enables you to:

1. Detect overfitting
2. Perform reliable hyperparameter tuning
3. Make informed decisions about model selection and deployment

Remember, the key is to strike a balance between train and validation set sizes to ensure accurate accuracy prediction on the test set.

Answer3:
        A good percentage to reserve for the validation set depends on the specific problem, dataset size, and model complexity. However, here are some general guidelines:

- For small datasets (<10,000 samples), reserve 20-30% for validation
- For medium datasets (10,000-100,000 samples), reserve 15-25% for validation
- For large datasets (>100,000 samples), reserve 10-20% for validation

These percentages aim to balance the trade-off between:

1. Having enough training data to learn patterns
2. Having enough validation data to accurately estimate test set accuracy

Reserving too little data for validation (e.g., 5%) may lead to overestimation of accuracy, while reserving too much (e.g., 50%) may waste valuable training data.

Remember, these are rough guidelines. You may need to adjust the validation set size based on your specific use case and model performance.

Also, consider using techniques like:

- Cross-validation (e.g., k-fold, stratified k-fold)
- Bootstrap sampling
- Walk-forward optimization

These methods can help you make the most of your data and improve model evaluation.

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Answer 1:
        Yes, averaging the validation accuracy across multiple splits can provide more consistent results. This technique is known as:

1. _Cross-validation_: Divide the data into k folds, train on k-1 folds, and validate on the remaining fold. Repeat this process k times, and average the validation accuracy.
2. _Bootstrap sampling_: Sample the data with replacement, train on the sampled data, and validate on the out-of-bag data. Repeat this process multiple times, and average the validation accuracy.

Averaging across multiple splits helps:

1. _Reduce variance_: Reduces the impact of a single split's variability on the validation accuracy.
2. _Improve robustness_: Provides a more robust estimate of the model's performance, less sensitive to individual split characteristics.
3. _Increase confidence_: Gives you more confidence in the model's performance, as the averaged accuracy is less prone to fluctuations.

Common cross-validation techniques include:

1. _k-fold cross-validation_ (e.g., 5-fold, 10-fold)
2. _Stratified k-fold cross-validation_ (maintains class balance)
3. _Leave-one-out cross-validation_ (extreme case, where k=n)

Bootstrap sampling can also be used for:

1. _Hyperparameter tuning_
2. _Model selection_
3. _Uncertainty estimation_

By averaging across multiple splits, you can obtain a more reliable estimate of your model's performance, which can lead to better decision-making and more effective model deployment.

Answer 2:
         [25/8, 1:44 pm] Meta AI: Yes, averaging the validation accuracy across multiple splits can provide a more accurate estimate of the test accuracy. This is because:

1. *Reduced overfitting*: By averaging across multiple splits, you reduce the impact of overfitting to a single split, which can lead to overly optimistic estimates of test accuracy.
2. *Better representation*: Averaging across multiple splits provides a more comprehensive representation of the model's performance on unseen data, as each split is an independent evaluation.
3. *Improved generalization*: By evaluating the model on multiple splits, you can assess its ability to generalize to new, unseen data, which is a key aspect of test accuracy.

Studies have shown that cross-validation and bootstrap sampling can provide a more accurate estimate of test accuracy compared to a single split. This is because:

1. *Reduced variance*: Averaging across multiple splits reduces the variance of the estimate, providing a more stable and reliable estimate of test accuracy.
2. *Increased precision*: By averaging across multiple splits, you can increase the precision of the estimate, providing a more accurate estimate of test accuracy.

However, it's important to note that:

1. *No guarantee*: Averaging across multiple splits does not guarantee a perfect estimate of test accuracy.
2. *Dependence on data*: The quality of the estimate still depends on the quality of
[25/8, 1:44 pm] Meta AI: Yes, averaging the validation accuracy across multiple splits can provide a more accurate estimate of the test accuracy. This is because:

1. *Reduced overfitting*: By averaging across multiple splits, you reduce the impact of overfitting to a single split, which can lead to overly optimistic estimates of test accuracy.
2. *Better representation*: Averaging across multiple splits provides a more comprehensive representation of the model's performance on unseen data, as each split is an independent evaluation.
3. *Improved generalization*: By evaluating the model on multiple splits, you can assess its ability to generalize to new, unseen data, which is a key aspect of test accuracy.

Studies have shown that cross-validation and bootstrap sampling can provide a more accurate estimate of test accuracy compared to a single split. This is because:

1. *Reduced variance*: Averaging across multiple splits reduces the variance of the estimate, providing a more stable and reliable estimate of test accuracy.
2. *Increased precision*: By averaging across multiple splits, you can increase the precision of the estimate, providing a more accurate estimate of test accuracy.

However, it's important to note that:

1. *No guarantee*: Averaging across multiple splits does not guarantee a perfect estimate of test accuracy.
2. *Dependence on data*: The quality of the estimate still depends on the quality of the data and the model's ability to generalize.

By using techniques like cross-validation and bootstrap sampling, you can increase the confidence in your estimate of test accuracy and make more informed decisions about your model's performance.

Answer 3:
       The number of iterations (or splits) in cross-validation or bootstrap sampling can impact the estimate of test accuracy. Here's what happens:

*Higher iterations:*

1. *Improved precision*: More iterations generally lead to a more precise estimate of test accuracy, as the average is calculated over a larger number of independent evaluations.
2. *Reduced variance*: Higher iterations reduce the variance of the estimate, making it more stable and reliable.
3. *Better convergence*: With more iterations, the estimate is more likely to converge to the true test accuracy.

*However:*

1. *Diminishing returns*: Beyond a certain point (e.g., 10-20 iterations), the gains in precision and reduced variance may become marginal.
2. *Increased computation*: Higher iterations require more computational resources and time, which may be a limiting factor.
3. *Overfitting to the validation set*: In some cases, too many iterations can lead to overfitting to the validation set, rather than the test set.

*Optimal number of iterations:*

1. *Depends on data size*: For smaller datasets, fewer iterations (5-10) may be sufficient, while larger datasets may benefit from more iterations (20-50).
2. *Depends on model complexity*: More complex models may require more iterations to converge.
3. *Depends on desired precision*: If high precision is required, more iterations may be necessary.

In summary, higher


Answer 4:
         While increasing iterations can help improve the estimate of test accuracy, it may not entirely compensate for a very small train or validation dataset. Here's why:

_Small train dataset:_

1. _Insufficient learning_: A small train dataset may not provide enough information for the model to learn effective patterns.
2. _Overfitting_: Increasing iterations may exacerbate overfitting to the small train dataset.

_Small validation dataset:_

1. _Unreliable estimate_: A small validation dataset may not provide a reliable estimate of test accuracy, even with more iterations.
2. _High variance_: Small validation datasets can lead to high variance in the estimate, which may not be fully mitigated by increasing iterations.

However, increasing iterations can still help in the following ways:

1. _Reducing variance_: More iterations can reduce the variance of the estimate, making it more stable.
2. _Improving precision_: Increased iterations can provide a more precise estimate of test accuracy, but this may be limited by the small dataset size.

To deal with very small datasets, consider:

1. _Data augmentation_: Artificially increase the dataset size by generating new samples through transformations or simulations.
2. _Transfer learning_: Leverage pre-trained models and fine-tune them on your small dataset.
3. _Active learning_: Selectively sample new data points to add to the dataset, focusing on the most informative or uncertain points.
4. _Ensemble methods_: Combine multiple models trained on different subsets of the data to improve overall performance.

Remember, increasing iterations can help, but it's essential to address the underlying limitations of small datasets.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.